<a href="https://colab.research.google.com/github/doodger/dataScience/blob/development/Credit_card.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Work in progress
This will be a data project on an imbalanced data set, for practice purposes.